# Install Dependencies

In [ ]:
!pip install transformers datasets scikit-learn

# Import Libraries

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load Dataset

In [14]:
file_path = "/content/drive/MyDrive/4th year project/Fine tune/Diseases_Symptoms.csv"
df = pd.read_csv(file_path)

df = df[['Name', 'Symptoms']]
df = df.dropna()

In [15]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


# Encode Labels

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Name'])  # Convert disease names to numerical labels

# Save label mappings for future use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [21]:
label_mapping

{'Abdominal Hernia': np.int64(0),
 'Abdominal Injury': np.int64(1),
 'Abscess': np.int64(2),
 'Abscess of Nose': np.int64(3),
 'Abscess of the Lung': np.int64(4),
 'Achalasia': np.int64(5),
 'Acinic cell carcinoma': np.int64(6),
 'Actinic Keratosis': np.int64(7),
 'Acute Bronchospasm': np.int64(8),
 'Acute Glaucoma': np.int64(9),
 'Acute Kidney Injury': np.int64(10),
 'Acute Otitis Media': np.int64(11),
 'Acute Pancreatitis': np.int64(12),
 'Acute Sinusitis': np.int64(13),
 'Adenoid cystic carcinoma': np.int64(14),
 'Adrenal Cancer': np.int64(15),
 'Alcohol Intoxication': np.int64(16),
 'Alcohol Use Disorder': np.int64(17),
 'Alopecia': np.int64(18),
 'Amyloidosis': np.int64(19),
 'Anal Fissure': np.int64(20),
 'Anemia': np.int64(21),
 'Anemia due to Chronic Kidney Disease': np.int64(22),
 'Anemia due to Malignancy': np.int64(23),
 'Anemia of Chronic Disease': np.int64(24),
 'Angle-Closure Glaucoma': np.int64(25),
 'Anorexia Nervosa': np.int64(26),
 'Antihypertensives Poisoning': np.in

# Split Dataset

In [22]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Symptoms'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

train_data = {'text': train_texts, 'label': train_labels}
val_data = {'text': val_texts, 'label': val_labels}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# Tokenization

In [23]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

# Data Collator

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation Metrics

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


# Load Model

In [26]:
num_labels = len(df['label'].unique())

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training Arguments

In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2
)


# Trainer Setup

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-29-2812472608.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Train the Model

In [30]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gamindapremasiri9 (gamindapremasiri9-university-of-kelaniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=200, training_loss=5.906983642578125, metrics={'train_runtime': 260.4756, 'train_samples_per_second': 6.143, 'train_steps_per_second': 0.768, 'total_flos': 17584385140608.0, 'train_loss': 5.906983642578125, 'epoch': 5.0})

# Save Model and Tokenizer

In [31]:
model.save_pretrained("/content/drive/MyDrive/4th year project/Fine tune/small-language-model")
tokenizer.save_pretrained("/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model")

('/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model/tokenizer_config.json',
 '/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model/special_tokens_map.json',
 '/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model/vocab.txt',
 '/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model/added_tokens.json',
 '/content/drive/MyDrive/4th year project/Fine tune/content/small-language-model/tokenizer.json')

# Load Model and Predict

In [34]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "/content/drive/MyDrive/4th year project/Fine tune/small-language-model"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [35]:
file_path = "/content/drive/MyDrive/4th year project/Fine tune/Diseases_Symptoms.csv"
df = pd.read_csv(file_path)

label_encoder = LabelEncoder()
label_encoder.fit(df['Name'])

LabelEncoder()

In [36]:
test_text = "Shortness of breath, Palpitations, Sweating"

In [38]:
import torch
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted label ID
logits = outputs.logits
predicted_label_id = torch.argmax(logits, dim=1).item()

probabilities = torch.nn.functional.softmax(logits, dim=1)
confidence = probabilities[0][predicted_label_id].item() * 100  # Convert to percentage

# Map label ID to disease name
predicted_disease = label_encoder.inverse_transform([predicted_label_id])[0]

print(f"Predicted Disease: {predicted_disease}")

Predicted Disease: Fat Embolism
